In [ ]:
def construct_splits_pereira(X, y, data_labels, alphas, feature_grouper, 
                             n_iter, use_kernelized, dataset, features_list, 
                             test_indices_arr, experiments, n_passages_384, 
                             n_passages_243, exp):
    
    # select which experiment we'll use for testing for this round of k-fold    
    for test_experiment in experiments:
        
        print(f"Test experiment: {test_experiment}")
        
        if test_experiment == '384':
            n_passages = n_passages_384
        else:
            n_passages = n_passages_243
        
        # select which passage we'll use for testing 
        for test_passage_number in n_passages: 
            
            test_indices = split_by_exp_passage_num(test_experiment, test_passage_number, data_labels)
            test_labels = data_labels[test_indices]
            train_indices = np.setdiff1d(np.arange(data_labels.shape[0]), test_indices)
            train_labels = data_labels[train_indices]
            
            test_indices_arr.extend(test_indices)
            
            # now we'll do k-fold validation for each of the remaining passage nums in the 
            # test experiment to determine the optimal alpha parameter 
            X_train = X[train_indices]
            y_train = y[train_indices]
            X_test = X[test_indices]
            y_test = y[test_indices]
            num_voxels = y_train.shape[1]

            if test_experiment == '384':
                n_passages = n_passages_384
                val_passages = np.concatenate((np.delete(n_passages, test_passage_number), n_passages_243))
                val_exp_names = np.concatenate((np.repeat('384', n_passages.shape[0]-1), 
                                        np.repeat('243', n_passages_243.shape[0])))
            else:
                n_passages = n_passages_243
                val_passages = np.concatenate((np.delete(n_passages, test_passage_number), n_passages_384))
                val_exp_names = np.concatenate((np.repeat('243', n_passages.shape[0]-1), 
                                        np.repeat('384', n_passages_384.shape[0])))
                
            if exp != 'both':
                idxs_exp = np.argwhere(val_exp_names==exp)
                val_passages = val_passages[idxs_exp].squeeze()
                val_exp_names = val_exp_names[idxs_exp].squeeze()

            mse_test, mse_intercept, val_perf, y_pred, y_pred_intercept = run_himalayas(X_train, 
                                            y_train, X_test, y_test, alphas, 
                                            train_labels, feature_grouper, n_iter, use_kernelized, 
                                            dataset, features_list, val_passages, val_exp_names)
        
            val_stored.append(val_perf)
        
            mse_stored_intercept_only.append(mse_intercept)
            mse_stored.append(mse_test)
            y_hat_folds.append(y_pred)
            y_test_folds.append(y_test)
            test_fold_size.append(X_test.shape[0])
            
            return mse_stored_intercept_only, mse_stored, y_hat_folds, y_test_folds, test_fold_size
            
    

In [ ]:
   
        if dataset == 'pereira':
           
        
        elif dataset == 'federonko':
            
            sentence_length = 8
            sentence_num = 52
            split_size = 32
            
            for i in range(0, sentence_num*sentence_length, split_size):
                
                test_indices = np.arange(i, i+split_size)
                train_indices = np.setdiff1d(np.arange(sentence_num*sentence_length), test_indices)
                test_labels = data_labels[test_indices]
                train_labels = data_labels[train_indices]

                X_train = X[train_indices]
                y_train = y[train_indices]
                X_test = X[test_indices]
                y_test = y[test_indices]
                
                mse_test, mse_intercept, val_perf, y_pred, y_pred_intercept = run_himalayas(X_train, 
                                                y_train, X_test, y_test, alphas, 
                                                train_labels, feature_grouper, n_iter, use_kernelized, 
                                                dataset, features_list, val_passages=None, val_exp_names=None)
            
                val_stored.append(val_perf)
                mse_stored_intercept_only.append(mse_intercept)
                mse_stored.append(mse_test)
                y_hat_folds.append(y_pred)
                y_test_folds.append(y_test) 
                test_fold_size.append(X_test.shape[0])   
            
        elif dataset == 'blank':
            
            num_samples = data_labels.shape[0]
            
            for test_story in np.unique(data_labels):
                
                test_indices = np.argwhere(data_labels==test_story)
                train_indices = np.setdiff1d(np.arange(num_samples), test_indices)
                test_labels = np.squeeze(data_labels[test_indices])
                train_labels = np.squeeze(data_labels[train_indices])
                
                X_train = np.squeeze(X[train_indices])
                y_train = np.squeeze(y[train_indices])
                X_test = np.squeeze(X[test_indices])
                y_test = np.squeeze(y[test_indices])
                
                mse_test, mse_intercept, val_perf, y_pred = run_himalayas(X_train, 
                                            y_train, X_test, y_test, alphas, 
                                            train_labels, feature_grouper, n_iter, use_kernelized, 
                                            dataset, features_list, val_passages=None, val_exp_names=None)
        
                
                val_stored.append(val_perf)
                
                mse_stored_intercept_only.append(mse_intercept)
                mse_stored.append(mse_test)
                y_hat_folds.append(y_pred)
                y_test_folds.append(y_test)
                test_fold_size.append(X_test.shape[0])